In [ ]:
# import torch
# import torch.nn as nn
# import torch.functional as F
# import pytorch_lightning as pl

# from torchrbpnet.layers import Conv1DFirstLayer, Conv1DResBlock, IndexEmbeddingOutputHead
# from torchrbpnet.losses import MultinomialNLLLossFromLogits
# from torchrbpnet.data.utils import TFIterableDataset

In [ ]:
# class Network(nn.Module):
#     def __init__(self, tasks, n_layers=9, n_body_filters=256):
#         super(Network, self).__init__()

#         self.tasks = tasks

#         self.body = nn.Sequential(*[Conv1DFirstLayer(4, n_body_filters, 6)]+[(Conv1DResBlock(n_body_filters, n_body_filters, dilation=(2**i))) for i in range(n_layers)])
#         self.head = IndexEmbeddingOutputHead(len(self.tasks), dims=n_body_filters)
    
#     def forward(self, inputs, **kwargs):
#         x = inputs

#         for layer in self.body:
#             x = layer(x)

#         return self.head(x)

# network = Network(tasks=list(range(223)))

In [ ]:
# class MultiRBPNet(pl.LightningModule):
#     def __init__(self, network):
#         super().__init__()
#         self.network = network # Network(tasks=range(223))
#         self.loss_fn = MultinomialNLLLossFromLogits()

#     def training_step(self, batch, **kwargs):
#         x, y = batch
#         y_pred = self.network(x)
#         loss = self.loss_fn(y, y_pred, dim=-2)
#         return loss

#     def configure_optimizers(self):
#         optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
#         return optimizer

# model = MultiRBPNet(network)
# model

In [ ]:
# dataset = TFIterableDataset('example-data-matrix/windows.chr13.4.data.matrix.filtered.tfrecord', shuffle=1_000_000)
# dataloader = torch.utils.data.DataLoader(dataset, batch_size=None)

In [1]:
import pytorch_lightning as pl
from pytorch_lightning import loggers as pl_loggers

/home/marc/miniconda3/envs/torch/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from torchrbpnet.data import tfrecord_to_dataloader, dummy_dataloader

# dataloader = tfrecord_to_dataloader('example-data-matrix/windows.chr13.4.data.matrix.filtered.tfrecord')
dataloader = dummy_dataloader(20)

2023-02-16 11:45:37.853049: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-16 11:45:38.701673: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-02-16 11:45:38.701722: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-02-16 11:45:38.701729: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [3]:
from torchrbpnet.networks import MultiRBPNet
# from torchrbpnet.lightning import Model

In [ ]:
# model = Model(network=MultiRBPNet(tasks=list(range(223))))
# model = Model(network=MultiRBPNet(tasks=list(range(7))))

In [ ]:
# %%
import torch
import torch.nn as nn
import pytorch_lightning as pl

from torchrbpnet.losses import MultinomialNLLLossFromLogits
from torchrbpnet.metrics import batched_pearson_corrcoef

# %%
class Model(pl.LightningModule):
    def __init__(self, network):
        super().__init__()
        self.network = network
        self.loss_fn = MultinomialNLLLossFromLogits()
        # self.metrics = [batched_pearson_corrcoef] + [self.loss_fn]

    def training_step(self, batch, *args, **kwargs):
        x, y = batch
        y_pred = self.network(x)
        loss = self.loss_fn(y, y_pred, dim=-2)
        metrics = self.compute_metrics(y, y_pred)
        self.log_dict(metrics, on_step=True, on_epoch=True, prog_bar=True)
        return loss

    def validation_step(self, batch, *args, **kwargs):
        x, y = batch
        y_pred = self.network(x)
        loss = self.loss_fn(y, y_pred, dim=-2)
        metrics = self.compute_metrics(y, y_pred, suffix='_val')
        self.log_dict(metrics, on_step=True, on_epoch=True, prog_bar=True)
        return loss

    # def compute_metrics(self, y, y_pred, suffix=''):
    #     metric_results = {f'{metric.__name__}' + suffix: metric(y, y_pred) for metric in self.metrics}
    #     return metric_results

    def compute_metrics(self, y, y_pred, suffix=''):
        pccs = batched_pearson_corrcoef(y, y_pred)
        return {'pcc_mean'+suffix: torch.mean(pccs), 'pcc_std'+suffix: torch.std(pccs)}

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

model = Model(network=MultiRBPNet(tasks=list(range(7))))

In [ ]:
from pytorch_lightning import loggers as pl_loggers

loggers = [
    pl_loggers.TensorBoardLogger('logs/tensorboard/'),
    pl_loggers.CSVLogger('logs/csv/'),
]

trainer = pl.Trainer(max_epochs=5, logger=loggers)
trainer.fit(model=model, train_dataloaders=dataloader, val_dataloaders=dataloader)